In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import git
import mlflow.keras
import os, sys
import numpy as np
from gensim.models.wrappers import FastText

import tensorflow_hub as hub
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.preprocessing import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import *

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
%matplotlib inline
pd.set_option('display.max_rows', None)

sys.path.append('../citation_bio_trainer')
#from feature.SpacyFeaturizer import get_spacy_feats_from_text
from feature.Featurizer import Featurizer
from util.Utils import calulate_ser_jer, load_from_folder, pad_sequences, load_embedding_matrix, evaluate, log_mlflow_results
import warnings
from model.FTLSTM import Metrics, calulate_ser_jer
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
# seed_value= 0

# import os
# os.environ['PYTHONHASHSEED']=str(seed_value)

# # 2. Set `python` built-in pseudo-random generator at a fixed value
# import random
# random.seed(seed_value)

# # 3. Set `numpy` pseudo-random generator at a fixed value
# import numpy as np
# np.random.seed(seed_value)

# # 4. Set the `tensorflow` pseudo-random generator at a fixed value
# import tensorflow as tf
# tf.compat.v1.set_random_seed(seed_value)

# # 5. Configure a new global `tensorflow` session
# from tensorflow.compat.v1.keras import backend as K
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# K.set_session(sess)

In [4]:
path = '/home/joshib/cs_data/citation-bio-labelled-data-2020-06-08~15:13:57.766608/nlp/exps/output/2020-06-08~15:13:57.766608'
#data_df = pd.read_csv(path+'/data-2020-06-08~15:14:01.185827.csv')

## LSTM with random embedding model

In [5]:
%%time
train_data_path = '/nlp/cs_mixed_20k/cs_mixed_20k_train/'
test_data_path  = '/nlp/cs_mixed_20k/cs_mixed_20k_test/'
eval_without_intra_newline_path  = '/nlp/eval_without_intra_newline/'
eval_with_intra_newline_path     = '/nlp/eval_with_intra_newline/'

sentences_train, sent_tags_train = load_from_folder(train_data_path)
sentences_test, sent_tags_test = load_from_folder(test_data_path)
sentences_eval1, sent_tags_eval1 = load_from_folder(eval_without_intra_newline_path)
#sentences_eval2, sent_tags_eval2 = load_from_folder(eval_with_intra_newline_path)

CPU times: user 1min 29s, sys: 878 ms, total: 1min 30s
Wall time: 1min 30s


In [6]:
xtrain, xvalid, ytrain, yvalid = train_test_split(sentences_train, sent_tags_train, test_size=0.1, random_state=42)

In [7]:
feat_config = {'max_vocab':100000,
               'lstm_feats':True, 
               'spacy_feats':True, 
               'google_feats': False, 
               'parscit_feats': True}

In [8]:
%%time
featurizer = Featurizer(feat_config)
print("loading train ...")
train_dict, tokenizer, maxlen = featurizer.fit_transform(xtrain, ytrain)
print("loading valid ...")
valid_dict = featurizer.transform(xvalid, yvalid)
print("loading test ...")
test_dict  = featurizer.transform(sentences_test, sent_tags_test)
print("loading evals ...")
eval_dict1  = featurizer.transform(sentences_eval1, sent_tags_eval1)
#eval_dict2  = featurizer.transform(sentences_eval2, sent_tags_eval2)

loading train ...
total partitions = 15
total partitions = 15
loading valid ...
total partitions = 1
loading test ...
total partitions = 1
loading evals ...
total partitions = 1
CPU times: user 13min 15s, sys: 28.8 s, total: 13min 43s
Wall time: 21min 38s


### Neural parscit features (temporary fix)

In [9]:
# %%time
# from feature.ParsCitLSTM import ParsCitLSTM
# c = {
#         "model_file": "/nlp/parscit/parscit-29-latest.h5",
#         "label_dict_file": "/nlp/parscit/labels.json",
#         "tfhub_model_dir": "/nlp/parscit/resource/"}
# model = ParsCitLSTM(model_config=c)

In [10]:
# %%time
# train_df = pd.DataFrame([])
# valid_df = pd.DataFrame([])
# test_df = pd.DataFrame([])
# eval1_df = pd.DataFrame([])
# eval2_df = pd.DataFrame([])

# train_df['text'] = np.array(xtrain, dtype='object')
# valid_df['text'] = np.array(xvalid, dtype='object')
# test_df['text'] = np.array(sentences_test, dtype='object')
# eval1_df['text'] = np.array(sentences_eval1, dtype='object')
# eval2_df['text'] = np.array(sentences_eval2, dtype='object')

In [11]:
%%time
# train_parscit = model.get_parscit_blocks(train_df)
# valid_parscit = model.get_parscit_blocks(valid_df)
# test_parscit  = model.get_parscit_blocks(test_df)
# eval1_parscit = model.get_parscit_blocks(eval1_df)
# eval2_parscit = model.get_parscit_blocks(eval2_df)
train_parscit = pd.read_pickle('/nlp/temp/train_parscit.pickle')
valid_parscit = pd.read_pickle('/nlp/temp/valid_parscit.pickle')
test_parscit = pd.read_pickle('/nlp/temp/test_parscit.pickle')
eval1_parscit = pd.read_pickle('/nlp/temp/eval1_parscit.pickle')
#eval2_parscit = pd.read_pickle('/nlp/temp/eval2_parscit.pickle')

CPU times: user 598 ms, sys: 865 ms, total: 1.46 s
Wall time: 1.46 s


In [12]:
def pad_dummy_feats(df, maxlen):
    parscit = df.copy()
    parscit_feats = list(parscit['parscit_feat']) 
    parscit_padded = []
    for ind in range(len(parscit_feats)):
        parscit_mask = np.zeros((maxlen, 14), dtype='int8')
        if len(parscit_feats[ind]) <= maxlen:
            parscit_mask[0:len(parscit_feats[ind]), :] = parscit_feats[ind][:]
        else:
            parscit_mask[:] = parscit_feats[ind][0:maxlen,:]
        parscit_padded.append(parscit_mask)
    parscit_arr = np.array([i.tolist() for i in parscit_padded])
    return parscit_arr

In [13]:
%%time
train_parscit_arr = pad_dummy_feats(train_parscit, maxlen)
valid_parscit_arr = pad_dummy_feats(valid_parscit, maxlen)
test_parscit_arr  = pad_dummy_feats(test_parscit, maxlen)
eval1_parscit_arr = pad_dummy_feats(eval1_parscit, maxlen)
#eval2_parscit_arr = pad_dummy_feats(eval2_parscit, maxlen)

CPU times: user 4min 5s, sys: 9 s, total: 4min 14s
Wall time: 4min 14s


## Keras with pre-trained fast text embedding

In [14]:
model_config = {'bi-lstm':True,
                'lstm': {'use':True, 'num': 1, 'units':50},
                'dense':{'use':False, 'num': 1, 'units':50},
                'trainable': True,
                'optimizer': Adam(
        learning_rate=0.001,
        beta_1=0.0,
        beta_2=0.0,
        epsilon=1e-05,
        amsgrad=False,
    ),
                'output_activation' : 'sigmoid', 
                'batch_size': 16,
                'aux_feats': {'use':True, 'dim':26}
                
               }

In [15]:
%%time
wiki_model = FastText.load_fasttext_format('/nlp/cc.en.300.bin')

CPU times: user 2min 20s, sys: 5.32 s, total: 2min 25s
Wall time: 2min 25s


In [16]:
%%time
embedding_matrix = load_embedding_matrix(wiki_model, feat_config['max_vocab'], tokenizer.word_index, 300)

Found 2000000 word vectors.
number of null word embeddings: 9769
CPU times: user 7.89 s, sys: 304 ms, total: 8.19 s
Wall time: 8.19 s


In [17]:
# %%time ### old code reported in mlflow
# input = Input(shape=(maxlen,))
# embed = Embedding(input_dim=feat_config['max_vocab'], input_length=maxlen, output_dim=300, weights=[embedding_matrix], trainable=model_config['trainable'])(input)
# lstm = LSTM(model_config['lstm_units'], return_sequences=True)(embed)
# aux_feats = Input(shape=(maxlen,26))
# conc = Concatenate([lstm, aux_feats])
# td = TimeDistributed(Dense(1, activation=model_config['output_activation']))(lstm)
# model = Model(inputs=[input, aux_feats], outputs=td)
# model.compile(optimizer=model_config['optimizer'],
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )
# print(model.summary())

In [28]:
conc

<tf.Tensor 'concatenate_1/Identity:0' shape=(None, 3861, 76) dtype=float32>

In [108]:
%%time
inp = Input(shape=(maxlen,), name='main_input')
embed = Embedding(input_dim=feat_config['max_vocab'], input_length=maxlen, output_dim=300, weights=[embedding_matrix], trainable=model_config['trainable'])(inp)
lstm = LSTM(model_config['lstm']['units'], return_sequences=True)(embed)
aux_feats = Input(shape=(maxlen,26), name='auxiliary_input')
conc = Concatenate()([lstm, aux_feats])
y =  Dense(512)(conc)
out = TimeDistributed(Dense(1, activation=model_config['output_activation'], name='output'))(y)
model = Model(inputs=[inp, aux_feats], outputs=out, name='multi-input-model')
model.compile(optimizer=model_config['optimizer'],
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print(model.summary())

Model: "multi-input-model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 3861)]       0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 3861, 300)    30000000    main_input[0][0]                 
__________________________________________________________________________________________________
lstm_16 (LSTM)                  (None, 3861, 50)     70200       embedding_16[0][0]               
__________________________________________________________________________________________________
auxiliary_input (InputLayer)    [(None, 3861, 26)]   0                                            
__________________________________________________________________________________

In [109]:
from tensorflow.keras import callbacks
import numpy as np
import json
import os
from sklearn.metrics import *
import pandas as pd
import numpy as np
import mlflow

def calulate_ser_jer(y_true, y_pred):
    """
    :param y_true:
    :param y_pred:
    :param keep_tag:
    :return:
    """
    tn, fp, fn, tp = confusion_matrix(y_true=y_true, y_pred=y_pred).ravel()
    ser = 1.0
    if (tp + fp) > 0.0:
        ser = fp / float(tp + fp)
    jer = 1.0
    if (tp + fn) > 0.0:
        jer = fn / float(tp + fn)
    return ser, jer

class Metrics(callbacks.Callback):
    def __init__(self, train_data, validation_data):
        super(Metrics, self).__init__()
        self.validation_data = validation_data
        self.train_data = train_data
        
    def on_train_begin(self, logs={}):
        self.val_ser = []
        self.val_jer = []
        
    def on_epoch_end(self, epoch, logs):
        vx = self.validation_data[0]
        vy = self.validation_data[1]
        vx = np.array(vx)
        pred_y = None
        pred_y = self.model.predict(vx)
        py = np.argmax(pred_y, axis=-1)
        #vy = np.argmax(vy, axis=-1)
        vy = np.array(vy).flatten()
        py = np.array(py).flatten()
        ser, jer = calulate_ser_jer(vy, py)
        print(ser, jer)
        self.val_ser.append(ser)
        self.val_jer.append(jer)
        logs["val_ser"] = ser
        logs["val_jer"] = jer
        print(f"— val_ser: {ser} — val_jer: {jer}")
        return 

In [110]:
metrics = Metrics([train_dict['lstm_feats'], train_dict['labels']], [valid_dict['lstm_feats'], valid_dict['labels']])

In [111]:
train_aux = np.concatenate((train_dict['spacy_num_feats'], train_parscit_arr), axis=-1)
valid_aux = np.concatenate((valid_dict['spacy_num_feats'], valid_parscit_arr), axis=-1)
test_aux  = np.concatenate((test_dict['spacy_num_feats'], test_parscit_arr), axis=-1)
eval1_aux = np.concatenate((eval_dict1['spacy_num_feats'], eval1_parscit_arr), axis=-1)
#eval2_aux = np.concatenate((eval_dict2['spacy_num_feats'], eval2_parscit_arr), axis=-1)

In [ ]:
%%time

early_stop= EarlyStopping(monitor='val_loss',patience=3,verbose=0,mode='min',restore_best_weights=False, min_delta=0.0001)
history = model.fit([train_dict['lstm_feats'], train_aux], train_dict['labels'], verbose=1, epochs=100, batch_size= model_config['batch_size'], \
                    validation_data=([valid_dict['lstm_feats'], valid_aux], valid_dict['labels']), callbacks=[early_stop], shuffle=False)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
axes[0].plot(history.history['accuracy'])
axes[0].plot(history.history['val_accuracy'])
axes[0].title.set_text('model accuracy')
axes[0].set_ylabel('accuracy')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].plot(history.history['loss'])
axes[1].plot(history.history['val_loss'])
axes[1].title.set_text('model loss')
axes[1].set_ylabel('loss')
axes[1].set_xlabel('epoch')
axes[1].legend(['train', 'val'], loc='upper left')
fig.tight_layout()

In [ ]:
print('done')

### Analysis on validation data

In [22]:
%%time
valid_probs = model.predict([valid_dict['lstm_feats'], valid_aux])
valid_probs = valid_probs.reshape(valid_probs.shape[0], valid_probs.shape[1])
valid_preds = np.where(valid_probs > 0.5, 1, 0)

CPU times: user 10.4 s, sys: 1.58 s, total: 12 s
Wall time: 7.18 s


In [23]:
%%time
result_valid = evaluate(valid_dict['labels'].tolist(), valid_preds.tolist())

CPU times: user 12.7 s, sys: 64.7 ms, total: 12.8 s
Wall time: 12.8 s


In [24]:
result_valid

{'count': 1758,
 'mean_ser': 0.002805715631041411,
 'mean_jer': 0.004535388148081998,
 'mean_acc': 0.9999575699234403,
 'num_mistakes': 221}

In [ ]:
count_valid = 0
for ind in range(len(valid_preds)):
    pred = valid_preds[ind]
    true = valid_dict['labels'][ind]
    if (true == pred).all():
        pass
    else:
        count_valid += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(fp_ind)
            print(ind, 'False positives:', fp_ind)
            for x in fp_ind:
                #print(x)
                print(np.array(xvalid[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                print(np.array(xvalid[ind].split(" "))[max(0, x-3):x+4])

### Analysis of test result

In [19]:
%%time
keras_model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/e814b3fd2d854c1a83c5998cea627b59/artifacts/models")
test_probs = keras_model.predict([test_dict['lstm_feats'], test_aux])
test_probs = test_probs.reshape(test_probs.shape[0], test_probs.shape[1])
test_preds = np.where(test_probs > 0.5, 1, 0)

CPU times: user 16.6 s, sys: 3.67 s, total: 20.3 s
Wall time: 14.4 s


In [20]:
%%time
result_test = evaluate(test_dict['labels'].tolist(), test_preds.tolist())

CPU times: user 14 s, sys: 8.08 ms, total: 14 s
Wall time: 14 s


In [21]:
result_test

{'count': 1953,
 'mean_ser': 0.003022608534831472,
 'mean_jer': 0.0037607236813184367,
 'mean_acc': 0.9999632651962401,
 'num_mistakes': 238}

In [27]:
result_test

{'count': 1953,
 'mean_ser': 0.003022608534831472,
 'mean_jer': 0.0037607236813184367,
 'mean_acc': 0.9999632651962401,
 'num_mistakes': 238}

In [ ]:
result_test

In [ ]:
tags = {'dataset':'cs_mixed_20k', 'data_split':'test'}
opt = model_config['optimizer']
model_config['optimizer'] = str(opt.get_config())
log_mlflow_results(model, result_test, feat_config, model_config, tags)

In [ ]:
count = 0
for ind in range(len(test_preds)):
    pred = test_preds[ind]
    true = new_y_test_enc[ind]
    if (true == pred).all():
        pass
    else:
        count += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(fp_ind)
            print(ind, 'False positives:', fp_ind)
            for x in fp_ind:
                #print(x)
                print(np.array(new_X_test[ind])[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                print(np.array(new_X_test[ind])[max(0, x-3):x+4])

### Evaluation of eval dataset

In [ ]:
#keras_model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/e42c50d933ff49fb826dfe5874fa6d51/artifacts/models")

In [23]:
eval_probs1 = keras_model.predict([eval_dict1['lstm_feats'], eval1_aux])
eval_probs1 = eval_probs1.reshape(eval_probs1.shape[0], eval_probs1.shape[1])
eval_preds1 = np.where(eval_probs1 > 0.5, 1, 0)
result_eval1 = evaluate(eval_dict1['labels'].tolist(), eval_preds1.tolist())

In [24]:
result_eval1

{'count': 1328,
 'mean_ser': 7.586536749363777e-05,
 'mean_jer': 0.0033384519576073984,
 'mean_acc': 0.9999648945432079,
 'num_mistakes': 68}

In [ ]:
eval_probs2 = model.predict([eval_dict2['lstm_feats'], eval_dict2['spacy_num_feats']])
eval_probs2 = eval_probs2.reshape(eval_probs2.shape[0], eval_probs2.shape[1])
eval_preds2 = np.where(eval_probs2 > 0.5, 1, 0)
result_eval2 = evaluate(eval_dict2['labels'].tolist(), eval_preds2.tolist())

In [40]:
### Visualize errors

In [ ]:
count_valid = 0
for ind in range(len(eval1_aux)):
    pred = eval_preds1[ind]
    true = eval_dict1['labels'][ind]
    if (true == pred).all():
        pass
    else:
        count_valid += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(fp_ind)
            print(ind, 'False positives:', fp_ind)
            for x in fp_ind:
                #print(x)
                print(np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                print(np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4])

In [ ]:
### Save eval results to file

In [60]:
min(len(sentences_eval1[ind].split(" ")), maxlen)

3861

In [41]:
ind = 0
eval_path_dict = {}
for fpath in os.listdir(eval_without_intra_newline_path):
    if fpath not in ['data-gen-config.json', 'data_generation_stats.csv'] and ".csv" in fpath:
        eval_path_dict[ind] = fpath
        ind += 1

In [66]:
min(len(sentences_eval1[75].split(" ")), maxlen)

3861

In [65]:
len(eval_preds1[75])

3861

In [71]:
eval_folder_path = '/nlp/eval_data_retok_predictions_f3a9cc61e30f460499b95a4c2b7957ae'
if not os.path.exists(eval_folder_path):
    os.makedirs(eval_folder_path)
for ind in range(len(eval_preds1)):
    #print(ind)
    df = pd.DataFrame([], columns=['x', 'y'])
    seq_len = min(len(sentences_eval1[ind].split(" ")), maxlen)
    df['x'] = sentences_eval1[ind].split(" ")[0:seq_len]
    df['y'] = eval_preds1[ind][0:seq_len]
    df.to_csv(os.path.join(eval_folder_path, eval_path_dict[ind][0:-4] + '_pred.csv'))

In [70]:
tags = {'dataset':'cs_mixed_20k', 'data_split':'eval_with_intra_newline_path'}
#opt = model_config['optimizer']
#model_config['optimizer'] = str(opt.get_config())
log_mlflow_results(keras_model, result_eval1, feat_config, model_config, tags)

2020/08/14 10:53:26 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under s3://caps-s3-mlflow/artifacts/4/f3a9cc61e30f460499b95a4c2b7957ae/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


In [ ]:
sentences_train[0][0:500]

In [ ]:
from feature.ParsCitLSTM import ParsCitLSTM

In [ ]:
#df = pd.read_csv('/nlp/parscit/data.csv')

In [ ]:
df = pd.DataFrame([])
df['text'] = np.array(sentences_train[0:100], dtype='object')

In [ ]:
%%time
df_out = model.dask_predict(df)

In [ ]:
#df_out

In [ ]:
# %%time
# import dask.dataframe as dd
# parscit_df = model.get_parscit_dask(df, blocksize=10)

In [ ]:
len(sentences_train[0].split(" "))

In [ ]:
%%time
x,y,z = model.predict(sentences_train[0])

In [ ]:
# for i,j in zip(x,y):
#     print(i,j)

### One predict test

In [ ]:
# model = load_model(c['model_file'])
# model._name='parscit_model'

In [ ]:
ls = np.random.randint(10, size=(1000))

In [ ]:
df = pd.DataFrame([], columns=['num'])
df['num'] = ls

In [ ]:
%%time
import pandas as pd
train_parscit = pd.read_pickle('/nlp/temp/train_parscit.pickle')
valid_parscit = pd.read_pickle('/nlp/temp/valid_parscit.pickle')
test_parscit = pd.read_pickle('/nlp/temp/test_parscit.pickle')
eval1_parscit = pd.read_pickle('/nlp/temp/eval1_parscit.pickle')
eval2_parscit = pd.read_pickle('/nlp/temp/eval2_parscit.pickle')

In [ ]:
def pad_dummy_feats(parscit, maxlen):
    parscit_feats = list(parscit['parscit_feat']) 
    parscit_padded = []
    for ind in range(len(parscit_feats)):
        parscit_mask = np.zeros((maxlen, 14), dtype='int8')
        if len(parscit_feats[ind]) <= maxlen:
            parscit_mask[0:len(parscit_feats[ind]), :] = parscit_feats[ind][:]
        else:
            parscit_mask[:] = parscit_feats[ind][0:maxlen,:]

        parscit_padded.append(parscit_mask)
    parscit_arr = np.array([i.tolist() for i in parscit_padded])
    return parscit_arr

In [ ]:
bla.shape

In [ ]:
bla1 = 